In [1]:
import pyspark
from pyspark.sql import SparkSession
import os

In [2]:
! apt install iputils-ping -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcap2-bin libpam-cap
The following NEW packages will be installed:
  iputils-ping libcap2-bin libpam-cap
0 upgraded, 3 newly installed, 0 to remove and 1 not upgraded.
Need to get 76.9 kB of archives.
After this operation, 280 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libcap2-bin amd64 1:2.44-1ubuntu0.22.04.2 [26.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 iputils-ping amd64 3:20211215-1ubuntu0.1 [43.0 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpam-cap amd64 1:2.44-1ubuntu0.22.04.2 [7930 B]
Fetched 76.9 kB in 2s (50.1 kB/s)   
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package libcap2-bin.
(Reading database ... 17759 files and directories currently ins

In [ ]:
! ping 10.107.38.26

PING 10.107.38.26 (10.107.38.26) 56(84) bytes of data.


In [2]:
## DEFINE VARIABLES
# CATALOG_URI = "http://nessie-service.nessie-dev.svc.cluster.local:19120/api/v1"
CATALOG_URI = "http://nessie-service.nessie-dev.svc.cluster.local:6788/api/v1"
# WAREHOUSE = "s3://iceberg/"
WAREHOUSE = "s3a://iceberg/"
# STORAGE_URI = "http://minio.minio-dev.svc.cluster.local:9000"
# STORAGE_URI = "http://minio-service.minio-dev.svc.cluster.local:6544"
# to define ip run: kubectl get svc -n minio-dev
STORAGE_URI = "http://10.107.38.26:6544"

In [3]:
MINIO_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID")
MINIO_SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
print(MINIO_ACCESS_KEY)

minio


In [4]:
## CONFIGURE SPARK SESSION
        # .set('spark.jars.packages', 
        #      'org.postgresql:postgresql:42.7.3,'
        #      'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0,'
        #      'org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.77.1,'
        #      'org.apache.hadoop:hadoop-aws:3.3.4,'
        #      'software.amazon.awssdk:bundle:2.24.8,'
        #      'software.amazon.awssdk:url-connection-client:2.24.8')             

conf = (
    pyspark.SparkConf()
        .setAppName('Iceberg Ingestion')
        .set('spark.sql.extensions', 
             'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,'
             'org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
        .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', CATALOG_URI)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        .set('spark.sql.catalog.nessie.s3.endpoint', STORAGE_URI)
        .set('spark.sql.catalog.nessie.warehouse', WAREHOUSE)
        .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
        .set("spark.hadoop.fs.s3a.endpoint", STORAGE_URI)
        .set("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY)
        .set("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY)
        .set("spark.hadoop.fs.s3a.path.style.access", "true")
        .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
)

In [5]:
## START SPARK SESSION
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/17 19:18:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Running


In [6]:
# Define the JDBC connection properties
jdbc_url = "jdbc:postgresql://postgres.psql-dev.svc.cluster.local:5432/postgres"
properties = {
    "user": "postgres",
    "password": "postgres",
    "driver": "org.postgresql.Driver"
}

In [7]:
sales_df = spark.read.jdbc(url=jdbc_url, table="fashion_sales", properties=properties)

In [8]:
sales_df.show()

+---+---------------+---------+------------+----------+--------------+------------------+-------------+
| id|   product_name| category|sales_amount|sales_date|store_location|customer_age_group|campaign_name|
+---+---------------+---------+------------+----------+--------------+------------------+-------------+
|  1| Slim Fit Jeans|    Denim|       89.99|2024-03-01|      New York|             18-24|Spring Launch|
|  2| Leather Jacket|Outerwear|      249.99|2024-03-01|   Los Angeles|             25-34|Spring Launch|
|  3|Graphic T-Shirt|     Tops|       39.99|2024-03-02|       Chicago|             18-24|March Madness|
|  4|   Summer Dress|  Dresses|      129.99|2024-03-03|      New York|             35-44|March Madness|
|  5|Casual Sneakers| Footwear|       99.99|2024-03-03|   Los Angeles|             25-34|Spring Launch|
+---+---------------+---------+------------+----------+--------------+------------------+-------------+



In [9]:
# output_path = "/workspace/out" 
# sales_df.write.parquet(output_path)

In [9]:
BUCKET_NAME = "src"
FILE_PATH = "data/parquet/part-00000.snappy.parquet"

s3a_path = f"s3a://{BUCKET_NAME}/{FILE_PATH}"

df = spark.read.parquet(s3a_path)

df.show()

26/02/17 19:19:05 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+---+---------------+---------+------------+----------+--------------+------------------+-------------+
| id|   product_name| category|sales_amount|sales_date|store_location|customer_age_group|campaign_name|
+---+---------------+---------+------------+----------+--------------+------------------+-------------+
|  1| Slim Fit Jeans|    Denim|       89.99|2024-03-01|      New York|             18-24|Spring Launch|
|  2| Leather Jacket|Outerwear|      249.99|2024-03-01|   Los Angeles|             25-34|Spring Launch|
|  3|Graphic T-Shirt|     Tops|       39.99|2024-03-02|       Chicago|             18-24|March Madness|
|  4|   Summer Dress|  Dresses|      129.99|2024-03-03|      New York|             35-44|March Madness|
|  5|Casual Sneakers| Footwear|       99.99|2024-03-03|   Los Angeles|             25-34|Spring Launch|
+---+---------------+---------+------------+----------+--------------+------------------+-------------+



In [10]:
# spark.sparkContext._jvm.org.apache.hadoop.util.VersionInfo.getVersion()

In [10]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.sales;")

DataFrame[]

In [11]:
sales_df.writeTo("nessie.sales.fashion_sales").createOrReplace()

In [12]:
spark.read.table("nessie.sales.fashion_sales").show()

+---+---------------+---------+------------+----------+--------------+------------------+-------------+
| id|   product_name| category|sales_amount|sales_date|store_location|customer_age_group|campaign_name|
+---+---------------+---------+------------+----------+--------------+------------------+-------------+
|  1| Slim Fit Jeans|    Denim|       89.99|2024-03-01|      New York|             18-24|Spring Launch|
|  2| Leather Jacket|Outerwear|      249.99|2024-03-01|   Los Angeles|             25-34|Spring Launch|
|  3|Graphic T-Shirt|     Tops|       39.99|2024-03-02|       Chicago|             18-24|March Madness|
|  4|   Summer Dress|  Dresses|      129.99|2024-03-03|      New York|             35-44|March Madness|
|  5|Casual Sneakers| Footwear|       99.99|2024-03-03|   Los Angeles|             25-34|Spring Launch|
+---+---------------+---------+------------+----------+--------------+------------------+-------------+



In [13]:
sales_df.writeTo("nessie.sales.fashion_sales_temp").createOrReplace()

In [14]:
spark.read.table("nessie.sales.fashion_sales_temp").show()

+---+---------------+---------+------------+----------+--------------+------------------+-------------+
| id|   product_name| category|sales_amount|sales_date|store_location|customer_age_group|campaign_name|
+---+---------------+---------+------------+----------+--------------+------------------+-------------+
|  1| Slim Fit Jeans|    Denim|       89.99|2024-03-01|      New York|             18-24|Spring Launch|
|  2| Leather Jacket|Outerwear|      249.99|2024-03-01|   Los Angeles|             25-34|Spring Launch|
|  3|Graphic T-Shirt|     Tops|       39.99|2024-03-02|       Chicago|             18-24|March Madness|
|  4|   Summer Dress|  Dresses|      129.99|2024-03-03|      New York|             35-44|March Madness|
|  5|Casual Sneakers| Footwear|       99.99|2024-03-03|   Los Angeles|             25-34|Spring Launch|
+---+---------------+---------+------------+----------+--------------+------------------+-------------+



In [15]:
spark.stop()